# Allstate Claims Severity

Kaggle challenge

# 1)- Importing key libraries


In [1]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

In [2]:
# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [3]:
# What's life without style :). So, let's add style to our dataframes
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [4]:
import pandas as pd
import numpy as np
from scipy import stats
import sys
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
# for modeling

# for modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV, LinearRegression, ElasticNetCV
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV, ElasticNet
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

# Regression evaluation
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 

# Classification
from sklearn.metrics import accuracy_score

#Model helper
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score

In [6]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas, numpy, seaborn, matplotlib,scipy,sklearn

Software versions
Python 3.6.3 64bit [MSC v.1900 64 bit (AMD64)]
IPython 7.4.0
OS Windows 10 10.0.16299 SP0
pandas 0.23.4
numpy 1.16.2
seaborn 0.9.0
matplotlib 2.1.0
scipy 1.1.0
sklearn 0.20.3
Tue Aug 27 18:42:18 2019 W. Europe Daylight Time

# 2)- Loading data

In [7]:
df_insurance=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_submit=pd.read_csv('sample_submission.csv')

In [8]:
df_insurance.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [9]:
df_insurance.tail()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
188313,587620,A,B,A,A,A,A,A,A,B,...,0.242437,0.289949,0.24564,0.30859,0.32935,0.223038,0.220003,0.333292,0.208216,1198.62
188314,587624,A,A,A,A,A,B,A,A,A,...,0.334270,0.382000,0.63475,0.40455,0.47779,0.307628,0.301921,0.318646,0.305872,1108.34
188315,587630,A,B,A,A,A,A,A,B,B,...,0.345883,0.370534,0.24564,0.45808,0.47779,0.445614,0.443374,0.339244,0.503888,5762.64
188316,587632,A,B,A,A,A,A,A,A,B,...,0.704364,0.562866,0.34987,0.44767,0.53881,0.863052,0.852865,0.654753,0.721707,1562.87
188317,587633,B,A,A,B,A,A,A,A,A,...,0.844563,0.533048,0.97123,0.93383,0.83814,0.932195,0.946432,0.810511,0.721460,4751.72


In [10]:
df_insurance.shape

(188318, 132)

In [11]:
df_insurance.columns

Index(['id', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9',
       ...
       'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12',
       'cont13', 'cont14', 'loss'],
      dtype='object', length=132)

In [12]:
df_test.shape

(125546, 131)

In [13]:
df_test.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In test set, we do not have our dependant feature i.e loss

### 2a)- Feature Description

In [14]:
df_insurance.describe()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
count,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000
mean,294135.982561,0.493861,0.507188,0.498918,0.491812,0.487428,0.490945,0.484970,0.486437,0.485506,0.498066,0.493511,0.493150,0.493138,0.495717,3037.337686
std,169336.084867,0.187640,0.207202,0.202105,0.211292,0.209027,0.205273,0.178450,0.199370,0.181660,0.185877,0.209737,0.209427,0.212777,0.222488,2904.086186
min,1.000000,0.000016,0.001149,0.002634,0.176921,0.281143,0.012683,0.069503,0.236880,0.000080,0.000000,0.035321,0.036232,0.000228,0.179722,0.670000
25%,147748.250000,0.346090,0.358319,0.336963,0.327354,0.281143,0.336105,0.350175,0.312800,0.358970,0.364580,0.310961,0.311661,0.315758,0.294610,1204.460000
50%,294539.500000,0.475784,0.555782,0.527991,0.452887,0.422268,0.440945,0.438285,0.441060,0.441450,0.461190,0.457203,0.462286,0.363547,0.407403,2115.570000
75%,440680.500000,0.623912,0.681761,0.634224,0.652072,0.643315,0.655021,0.591045,0.623580,0.566820,0.614590,0.678924,0.675759,0.689974,0.724623,3864.045000
max,587633.000000,0.984975,0.862654,0.944251,0.954297,0.983674,0.997162,1.000000,0.980200,0.995400,0.994980,0.998742,0.998484,0.988494,0.844848,121012.250000


### 2b)drop id feature

In [15]:
df_insurance.drop('id',axis=1, inplace=True)

In [16]:
df_insurance.shape

(188318, 131)

### 2c)- Data type

In [17]:
# Checking all variables with their data-type
def _tbl_dtype(dataset):
    sum_dtype = pd.DataFrame(dataset.dtypes).sort_values(0).rename(columns = {0:'Data Type'})
    return sum_dtype

table_dtype = _tbl_dtype(df_insurance)
table_dtype

,Data Type
loss,float64
cont1,float64
cont2,float64
cont3,float64
cont4,float64
cont5,float64
cont6,float64
cont14,float64
cont8,float64
cont9,float64


### 2d)-Skewness of the distribution

In [18]:
print(df_insurance.skew())

cont1     0.516424
cont2    -0.310941
cont3    -0.010002
cont4     0.416096
cont5     0.681622
cont6     0.461214
cont7     0.826053
cont8     0.676634
cont9     1.072429
cont10    0.355001
cont11    0.280821
cont12    0.291992
cont13    0.380742
cont14    0.248674
loss      3.794958
dtype: float64


# 3)- Model preparing

In [19]:
# using already saved data

df_insurance=pd.read_pickle('train.pkl')
df_insurance.shape

(188318, 132)

In [20]:
df_insurance.head(2)

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,0,1,0,1,0,0,0,0,1,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,0,1,0,0,0,0,0,0,1,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60


In [21]:
#drop id variable from current data
df_insurance.drop('id',axis=1, inplace=True)
df_insurance.shape

(188318, 131)

### 3.1)- splitting x and y variables

In [22]:
Y = df_insurance['loss'].values
X = df_insurance.drop('loss', axis=1)
print(Y.shape)
print(X.shape)

(188318,)
(188318, 130)


### 3.2)-Split the data into train-test

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=2019)

# 4)- Linear Models

- Linear Regression
- Ridge Regression
- LASSO Regression
- Elastic Net Regression

### 4.1)- Linear Regression

OLS

In [24]:
model_lr = LinearRegression(n_jobs=-1)
model_lr.fit(X_train , y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [25]:
y_pred = model_lr.predict(X_test)

In [26]:
y_pred[:5]

array([4941.66896541, 7162.75235243, 7016.71094822, 6311.75695133,
       5442.28259355])

In [27]:
y_test[:5]

array([2129.69, 8289.68, 4606.95, 4103.9 , 2958.38])

In [28]:
mse_lr = mean_squared_error(y_test, y_pred)
mae_lr = mean_absolute_error(y_test, y_pred)
rmse_lr = mean_squared_error(y_test, y_pred)**0.5
r2_lr = r2_score(y_test, y_pred)

In [29]:
print('MSE    : %0.4f ' % mse_lr)
print('MAE    : %0.4f ' % mae_lr)
print('RMSE   : %0.4f ' % rmse_lr)
print('R2     : %0.4f ' % r2_lr)

MSE    : 4540104.0427 
MAE    : 1328.1657 
RMSE   : 2130.7520 
R2     : 0.4720 


### 4.2)- Ridge Regression

In [30]:
alpha=1.0 
model_ridge = Ridge(alpha=alpha,random_state=2019)
model_ridge.fit(X_train, y_train)
pred_ridge = model_ridge.predict(X_test)

In [31]:
pred_ridge[:5]

array([4938.88925098, 7160.09773944, 7019.79103491, 6311.04245425,
       5440.02496168])

In [32]:
y_test[:5]

array([2129.69, 8289.68, 4606.95, 4103.9 , 2958.38])

In [33]:
mse_ridge = mean_squared_error(y_test, pred_ridge)
mae_ridge = mean_absolute_error(y_test, pred_ridge)
rmse_ridge = mean_squared_error(y_test, pred_ridge)**0.5
r2_ridge = r2_score(y_test, pred_ridge)

In [34]:
print('MSE    : %0.4f ' % mse_ridge)
print('MAE    : %0.4f ' % mae_ridge)
print('RMSE   : %0.4f ' % rmse_ridge)
print('R2     : %0.4f ' % r2_ridge)

MSE    : 4539549.4635 
MAE    : 1328.0566 
RMSE   : 2130.6218 
R2     : 0.4721 


### 4.3)-RidgeCV

In [35]:
model_RidgeCV = RidgeCV(alpha=alpha,random_state=2019)
model_RidgeCV.fit(X_train, y_train)
pred_RidgeCV = model_RidgeCV.predict(X_test)

TypeError: __init__() got an unexpected keyword argument 'alpha'

In [ ]:
pred_RidgeCV[:5]

In [ ]:
y_test[:5]

In [ ]:
mse_ridgeCV = mean_squared_error(y_test, pred_RidgeCV)
mae_ridgeCV = mean_absolute_error(y_test, pred_RidgeCV)
rmse_ridgeCV = mean_squared_error(y_test, pred_RidgeCV)**0.5
r2_ridgeCV = r2_score(y_test, pred_RidgeCV)

In [ ]:
print('MSE    : %0.4f ' % mse_ridgeCV)
print('MAE    : %0.4f ' % mae_ridgeCV)
print('RMSE   : %0.4f ' % rmse_ridgeCV)
print('R2     : %0.4f ' % r2_ridgeCV)

### 4.4)-LASSO Linear Regression

In [ ]:
model_lasso = Lasso(alpha=alpha,random_state=2019)
model_lasso.fit(X_train, y_train)
pred_lasso = model_lasso.predict(X_test)

In [ ]:
mse_lasso = mean_squared_error(y_test, pred_lasso)
mae_lasso = mean_absolute_error(y_test, pred_lasso)
rmse_lasso = mean_squared_error(y_test, pred_lasso)**0.5
r2_lasso = r2_score(y_test, pred_lasso)

In [ ]:
print('MSE    : %0.4f ' % mse_lasso)
print('MAE    : %0.4f ' % mae_lasso)
print('RMSE   : %0.4f ' % rmse_lasso)
print('R2     : %0.4f ' % r2_lasso)

### 4.5)-lassoCV

In [ ]:
model_lassoCV=LassoCV(random_state=2019)
model_lassoCV.fit(X_train,y_train)
pred_lassoCV = model_lassoCV.predict(X_test)

In [ ]:
mse_lassoCV = mean_squared_error(y_test, pred_lassoCV)
mae_lassoCV = mean_absolute_error(y_test, pred_lassoCV)
rmse_lassoCV = mean_squared_error(y_test,pred_lassoCV)**0.5
r2_lassoCV = r2_score(y_test, pred_lassoCV)

In [ ]:
print('MSE    : %0.4f ' % mse_lassoCV)
print('MAE    : %0.4f ' % mae_lassoCV)
print('RMSE   : %0.4f ' % rmse_lassoCV)
print('R2     : %0.4f ' % r2_lassoCV)

### 4.6)-LassoLarsCV

In [ ]:
model_LassoLarsCV=LassoLarsCV()
model_LassoLarsCV.fit(X_train,y_train)
pred_lassolarsCV = model_LassoLarsCV.predict(X_test)

In [ ]:
mse_lassolars = mean_squared_error(y_test, pred_lassolarsCV)
mae_lassolars = mean_absolute_error(y_test, pred_lassolarsCV)
rmse_lassolars = mean_squared_error(y_test, pred_lassolarsCV)**0.5
r2_lassolars = r2_score(y_test, pred_lassolarsCV)

In [ ]:
print('MSE    : %0.4f ' % mse_lassolars)
print('MAE    : %0.4f ' % mae_lassolars)
print('RMSE   : %0.4f ' % rmse_lassolars)
print('R2     : %0.4f ' % r2_lassolars)

### 4.7)-# ElasticNet Regression

In [ ]:
model_ElasticNet=ElasticNet()
model_ElasticNet.fit(X_train,y_train)
pred_ElasticNet = model_ElasticNet.predict(X_test)

In [ ]:
mse_ElasticNet = mean_squared_error(y_test, pred_ElasticNet)
mae_ElasticNet = mean_absolute_error(y_test, pred_ElasticNet)
rmse_ElasticNet = mean_squared_error(y_test, pred_ElasticNet)**0.5
r2_ElasticNet = r2_score(y_test, pred_ElasticNet)

In [ ]:
print('MSE    : %0.4f ' % mse_ElasticNet)
print('MAE    : %0.4f ' % mae_ElasticNet)
print('RMSE   : %0.4f ' % rmse_ElasticNet)
print('R2     : %0.4f ' % r2_ElasticNet)

### 4.8)-ElasticNetCV Regression

In [ ]:
model_ElasticNetCV=ElasticNetCV()
model_ElasticNetCV.fit(X_train,y_train)
pred_ElasticNetCV = model_ElasticNetCV.predict(X_test)

In [ ]:
mse_ElasticNetCV = mean_squared_error(y_test, pred_ElasticNetCV)
mae_ElasticNetCV = mean_absolute_error(y_test, pred_ElasticNetCV)
rmse_ElasticNetCV = mean_squared_error(y_test, pred_ElasticNetCV)**0.5
r2_ElasticNetCV = r2_score(y_test, pred_ElasticNetCV)

In [ ]:
print('MSE    : %0.4f ' % mse_ElasticNetCV)
print('MAE    : %0.4f ' % mae_ElasticNetCV)
print('RMSE   : %0.4f ' % rmse_ElasticNetCV)
print('R2     : %0.4f ' % r2_ElasticNetCV)